# Readout Cavity Calibration
*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
This tutorial introduces the simulation of readout cavity calibration using the readout simulator. The outline of this tutorial is as follows:
- Introduction
- Preparation
- Calibrating the Readout Cavity Transition Frequencies
- Calibrating the Dispersive Shift and Coupling Strength
- Measuring the decay rate
- Summary

## Introduction

In superconducting circuit, to acquire the state of a qubit, we can probe the readout cavity coupled with this qubit to achieve the qubit state indirectly. Concretely, we first apply readout pulse signal and then detect and analyze the reflected signal. Because the phase shift and amplitude change depend on the qubit state, we are able to know whether the outcome is "0" or "1" by this change.

In the real experiment of calibration, the first step to is to find the parameters of readout cavity. This tutorial introduces how to use Quanlse to simulate the readout cavity calibration.

A coupled cavity-qubit system can be described by the Jaynes-Cummings Hamiltonian in the dispersive regime \[1\]:

$$
\hat{H}_{\rm JC} = \omega_r \hat{a}^\dagger \hat{a} + \frac{1}{2}\omega_q \hat{\sigma}_z + \chi \hat{a}^\dagger \hat{a} \hat{\sigma}_z,
$$


where $\hat{a}$, $\hat{a}^\dagger$ are annihilation and creation operators and $\hat{\sigma}_z$ is the Pauli-Z operator. $\omega_r$ and $\omega_q$ denote the bare frequencies of the readout cavity and the qubit, $\chi$ is the dispersive shift and takes the form \[2\]:

$$
\chi = \frac{g^2 \alpha}{\Delta_{qr}(\Delta_{qr} + \alpha)}.
$$

where $\alpha$ is the qubit anharmonicity, $\Delta_{qr} = \omega_q - \omega_r$ is qubit-cavity detuning and $g$ is the qubit-cavity coupling strength. The interaction term $\chi \hat{a}^\dagger \hat{a} \hat{\sigma}_z$ in $\hat{H}_{\rm JC}$ gives rise to a shift of $2\chi$ in the transition frequency of the readout cavity when the qubit state is $|0\rangle$ and $|1\rangle$. Therefore in the experiment, by performing frequency sweep for the cavity with qubit state prepared in $|0\rangle$ or $|1\rangle$ respectively, we can obtain transition frequency $f_0$ and $f_1$ and therefore the frequency shift $2\chi$. Finally, the cavity-qubit couping strength is indirectly calculated using the expression above. 

In addition to the transition frequency and dispersive shift, the linewidth $\kappa$ can be measured to determine the photon decay rate of the readout cavity. To simulate the interaction between cavity-qubit system and the environment, the evolution of the system density matrix $\hat{\rho}(t)$ is given by Lindblad master equation \[3, 4\]:

$$
\frac{d \hat{\rho}(t)}{dt} = -i[\hat{H}(t), \hat{\rho}(t)] + \frac{\kappa}{2}[2 \hat{a} \hat{\rho}(t) \hat{a}^\dagger - \hat{\rho}(t) \hat{a}^\dagger \hat{a} - \hat{a}^\dagger \hat{a} \hat{\rho}(t)].
$$


The decay rate is therefore acquired by fitting the spectrum and extract the linewidth in the Lorentzian function.

The observable quantities we take are the two orthogonal quadratures $\hat{X} = \frac{1}{2}(\hat{a}^\dagger + \hat{a})$ and $\hat{Y} = \frac{i}{2}(\hat{a}^\dagger - \hat{a})$. In the experiment, through a series of signal processing on the pulse reflected from the readout cavity, we can obtain the voltage $V_I$ and $V_Q$ related to these two orthogonal quadratures.

In this tutorial, we simulate the calibration of readout cavity by solving the qubit-cavity dynamics: the cavity transition frequencies with different qubit states ($|0\rangle$ and $|1\rangle$) $\omega_{r0}$ and $\omega_{r1}$, the linewidth $\kappa$ and the dispersive shift $\chi$.

## Preparation

To run this tutorial we need to import the following necessary packages from Quanlse and other commonly-used Python libraries.

In [ ]:
# Import tools from Quanlse
from Quanlse.Simulator.ReadoutSim3Q import readoutSim3Q
from Quanlse.Calibration.Readout import resonatorSpec, fitLorentzian, lorentzian

# Import tools from other python libraries
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
from math import pi

## Calibrating the Readout Cavity Transition Frequencies

In this section, we will calibrate the transition frequencies of the reading cavity when the qubit is in the ground state and the first excited state, respectively. Here, we first use the predefined function `readoutSim3Q()` to return the Class object `readoutModel` containing information of the readout cavity.

In [ ]:
readoutModel = readoutSim3Q()  # Initialize a readoutModel object

Then, we set the range of frequency sweep `freqRange`, the drive amplitude `amp` and the duration of the readout pulse `duration`.

In [ ]:
freqRange = np.linspace(7.105, 7.125, 60) * 2 * pi  # the range of frequency to probe the resonator, in 2 pi GHz
amp = 0.0005 * (2 * pi)  # drive amplitude, in 2 pi GHz
duration = 1000  #  duration of the readout pulse, in nanoseconds

Use the function `resonatorSpec` to simulate the frequency sweep of the readout cavity when qubit is in the ground state, and input the index of the resonator `onRes`, the range of the frequency sweep `freqRange`, the amplitude `amp` and the duration `duration` with `qubitState` set to be in the ground state.

In [ ]:
vi0, vq0 = resonatorSpec(readoutModel=readoutModel, onRes=[0], freqRange=freqRange, 
                         amplitude=amp, duration=duration, qubitState='ground')

The result returns the measured signal $V_I$ and $V_Q$. We plot $V_Q$ (or $V_I$) with respect to the drive frequency.

In [ ]:
idx0 = find_peaks(vq0[0], height=max(vq0[0]))[0]  # find the index of the transition frequency
w0 = freqRange[idx0][0]  # transition frequency
print(f'The resonator transition frequency with qubit in ground state is {(w0 / (2 * pi)).round(3)} GHz')

In [ ]:
plt.plot(freqRange / (2 * pi), np.array(vq0[0]))
plt.plot()
plt.xlabel('$\omega_d$ (GHz)')
plt.ylabel('signal (a.u.)')
plt.title('Readout resonator spectrum')
plt.vlines((freqRange / (2 * pi))[idx0], 0, max(vq0[0]), linestyles='dashed')
plt.show()

From the result of the simulation shown above, we can see that the read cavity transition frequency is around 7.118 GHz when the qubit is in the ground state. Next, we calibrate the read cavity transition frequency when the qubit is in the excited state using the same procedure.

In [ ]:
vi1, vq1 = resonatorSpec(readoutModel=readoutModel, onRes=[0], freqRange=freqRange, 
                         amplitude=amp, duration=duration, qubitState='excited')

In [ ]:
idx1 = find_peaks(vq1[0], height=max(vq1[0]))[0]
w1 = freqRange[idx1][0]
print(f'The resonator transition frequency with qubit in excited state is {(w1 / (2 * pi)).round(3)} GHz')

In [ ]:
plt.plot(freqRange / (2 * pi), np.array(vq1[0]))
plt.plot()
plt.xlabel('$\omega_d$ (GHz)')
plt.ylabel('signal (a.u.)')
plt.title('Readout resonator spectrum')
plt.vlines((freqRange / (2 * pi))[idx1], 0, max(vq1[0]), linestyles='dashed')
plt.show()

It can be seen in the spectrum that the readout cavity transition frequency is about 7.112 GHz when the qubit is in the first excited state.

## Calibrating the Dispersive Shift and Coupling Strength 

In the previous section, we obtained the calibrated frequencies $f_0$ and $f_1$, so that the dispersion shift $\chi$ can be calculated directly by,

$$
\chi = \frac{|f_0 - f_1|}{2}.
$$

In [ ]:
chi = abs(w0 - w1) / 2
print(f'The dispersive shift is {(chi * 1e3 / (2 * pi)).round(3)} MHz')

Combining the expressions of $\chi$ given in the "Introduction" section, we can derive the expression of cavity-qubit coupling strength in terms of other known parameters:

$$
g = \sqrt{\frac{\chi\Delta_{qr}(\Delta_{qr}+\alpha)}{\alpha}}.
$$

Extract the theoretical parameters from `readoutModel` and calculate the coupling strength $g$ given above.

In [ ]:
# Extract parameters from the model
g = readoutModel.coupling[0]  # therotical qubit-resonator coupling strength 
wq = readoutModel.pulseModel.qubitFreq[0]  # qubit bare frequency
alpha = readoutModel.pulseModel.qubitAnharm[0]  # qubit anharmonicity
wr = (w0 + w1) / 2  # estimated resonator frequency
detuning = wq - wr  # qubit-resonator detuning

In [ ]:
# coupling strength calculation

def qrCoupling(chi, detuning, alpha):
    g = np.sqrt(abs(chi * detuning * (detuning + alpha) / alpha))
    return g

In [ ]:
gEst = qrCoupling(chi, detuning, alpha)  # Estimated qubit-resonator coupling strength

Compare the theoretical value and the estimated value of $g$.

In [ ]:
print(f'Theoretical coupling strength is {g * 1e3 / (2 * pi)} MHz')
print(f'Estimated coupling strength is {(gEst * 1e3 / (2 * pi)).round(1)} MHz')

The coupling strength of the readout cavity and the qubit, obtained by calibrating the dispersion shift and indirect calculations, is 135 MHz, which is in good agreement with the theoretical value of 134.0 MHz.

## Measuring the decay rate

After we have the spectrum of cavity frequency, we are able to estimate the decay rate $\kappa$ by the linewidth of the Lorentzian function. Here, we use the function `fitLorentzian`, input the range of frequency sweep and the reflected signal to fit the spectrum and estimate the linewidth $\kappa$.

In [ ]:
param, cov = fitLorentzian(freqRange, vq0[0])  # Fit the curve using lorentzian function
kappaEst = abs(param[2])  # Estimated linewidth

In [ ]:
plt.plot(freqRange / (2 * pi), lorentzian(freqRange, param[0], param[1], param[2], param[3]), '.')
plt.plot(freqRange / (2 * pi), vq0[0])
plt.xlabel('$\omega_d$ (GHz)')
plt.ylabel('signal (a.u.)')
plt.show()

Compare the theoretical value and the estimated value of decay rate (or linewidth).

In [ ]:
kappa = readoutModel.dissipation

print(f'Theoretical linewidth is {kappa * 1e3 / (2 * pi)} MHz')
print(f'Estimated linewidth is {(kappaEst * 1e3 / (2 * pi)).round(3)} MHz')

From the simulation results, we can see that the decay rate $\kappa$ set in the master equation is 2.0 MHz, while the linewidth obtained from the spectrum is 1.987 MHz, indicating that the interaciton strength between the reading cavity and the environment can be indirectly calibrated by scanning the frequency of the reading cavity and calculating the line width in the experiment.

## Summary
Users can click on this link [tutorial-readout-cavity-calibration.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/EN/tutorial-readout-cavity-calibration.ipynb) to jump to the corresponding GitHub page for this Jupyter Notebook documentation and run this tutorial. You can try different hardware parameters of the readout cavity and run the codes in this tutorial to simulate the cavity calibration in the superconducting quantum computing experiment.

## Reference

\[1\] [Blais, Alexandre, et al. "Cavity quantum electrodynamics for superconducting electrical circuits: An architecture for quantum computation." *Physical Review A* 69.6 (2004): 062320.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.69.062320)

\[2\] [Koch, Jens, et al. "Charge-insensitive qubit design derived from the Cooper pair box." *Physical Review A* 76.4 (2007): 042319.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.76.042319)

\[3\] [Lindblad, Goran. "On the generators of quantum dynamical semigroups." *Communications in Mathematical Physics* 48.2 (1976): 119-130.](https://link.springer.com/article/10.1007/bf01608499)

\[4\] [Bianchetti, R., et al. "Dynamics of dispersive single-qubit readout in circuit quantum electrodynamics." *Physical Review A* 80.4 (2009): 043840.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.80.043840)